# SQL Avancé

Nous utiliserons le dialecte Transact-SQL (T-SQL) : [https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15](https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15)  

## Révisions

\- Schémas

\- Création de table

> \- Clé primaire
> 
> \- Clé étrangère
> 
> \- Unicité

## Création de base

In [ ]:
-- On supprime et on re-créé tout
IF (EXISTS (SELECT [name] FROM [master].[sys].[databases] WHERE [name] = 'Kiosque'))
    DROP DATABASE [Kiosque]

CREATE DATABASE [Kiosque]

GO

USE [Kiosque]

## Schémas

\- ref

\- data

Un schéma -\> regroupement d'éléments SQL (Tables, Vuews, ..) pour faciliter la gestion de permissions

In [ ]:
CREATE SCHEMA [data]

GO

CREATE SCHEMA [ref]

GO

## Création de Tables

In [ ]:
CREATE TABLE [data].[Utilisateur_UTL] (
    [Id] UNIQUEIDENTIFIER NOT NULL PRIMARY KEY DEFAULT NEWID(),
    [Login] NVARCHAR(50) NOT NULL UNIQUE,
    [Email] NVARCHAR(255) NOT NULL UNIQUE,
    [Actif] BIT NOT NULL DEFAULT 0
)

In [ ]:
CREATE TABLE [ref].[Role_ROL] (
    [Id] TINYINT NOT NULL PRIMARY KEY IDENTITY(1, 1),
    [Key] CHAR(10) NOT NULL,
    [Description] NVARCHAR(100) NOT NULL
)

In [ ]:
CREATE TABLE [data].[RoleUtilisateur_RUL] (
    [ROL_Id] TINYINT NOT NULL,
    [UTL_Id] UNIQUEIDENTIFIER NOT NULL,
    CONSTRAINT [pk_rol_utl] PRIMARY KEY ([ROL_Id], [UTL_Id]),
    CONSTRAINT [fk_RUL_ROL] FOREIGN KEY ([ROL_Id]) REFERENCES [ref].[Role_ROL]([Id]),
    CONSTRAINT [fk_RUL_UTL] FOREIGN KEY ([UTL_Id]) REFERENCES [data].[Utilisateur_UTL]([Id])
)

## Modification de tables

In [ ]:
ALTER TABLE [data].[Utilisateur_UTL]
    ADD [MotDePasse] NVARCHAR(512) NULL

In [ ]:
CREATE TABLE [ref].[Permission_PER] (
    [Id] INT IDENTITY(1,1) NOT NULL,
    [Nom] NVARCHAR(25) NOT NULL,
    [Description] NVARCHAR(256) NULL
)

In [ ]:
ALTER TABLE [ref].[Permission_PER]
    ADD CONSTRAINT [PK_PER] PRIMARY KEY ([Id])

## Opréations CRUD

Create, Read, Update, Delete

  

Create -\>  INSERT

Read -\> SELECT

Update -\> UPDATE

Delete -\> DELETE

In [ ]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('admin', N'Administrateur'),
    ('guest', N'Visiteur'),
    ('market', N'Marketing')

In [ ]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('guest', N'Autre visiteur')

In [ ]:
SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

In [ ]:
DELETE FROM [ref].[Role_ROL]
WHERE [Id] IN (
    SELECT [Id]FROM [ref].[Role_ROL]
    WHERE [Description] = N'Autre visiteur'
)

SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

In [ ]:
ALTER TABLE [ref].[Role_ROL]
    ADD [ROL_Key] AS LOWER([Key]),
    CONSTRAINT [uq_key] UNIQUE([ROL_Key])

In [ ]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('guest', N'Autre visiteur')

In [ ]:
UPDATE [ref].[Role_ROL]
SET [Description] = CONCAT(N'Utilisateur ', LOWER([Description]))

SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

In [ ]:
INSERT INTO [data].[Utilisateur_UTL] ([Login], [Email], [Actif])
VALUES
    (N'alice', N'alice@u-picardie.fr', 1),
    (N'bob', N'bob@u-picardie.fr', 1),
    (N'carl', N'carl@u-picardie.fr', 0)

GO

DECLARE @UtlId UNIQUEIDENTIFIER
DECLARE @RolId TINYINT

SELECT TOP 1 @UtlId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'alice'

SELECT TOP 1 @RolId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'admin'

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @UtlId AS [UTL_Id], @RolId AS [ROL_Id]


Pour la prochaine séance :

\- Ajouter le rôle Marketing à Alice

\- Ajouter le rôle Marketing à Bob

\- Ajouter le rôle Guest à Carl